# Using Requests to interact with the GitHub API

This is a little introduction to using the [Requests](http://docs.python-requests.org/en/master/) library to interact with the [GitHub API](https://developer.github.com/v3/). The documentation for each is excellent, so the idea here is to give a few flavours of how they interact with one-another, and not to redundantly document the tools themselves.

First, import some useful functionality:

In [5]:
from pprint import pprint
from pathlib import Path

import requests

Next, get hold of a [GitHub personal access token](https://help.github.com/articles/creating-a-personal-access-token-for-the-command-line/) with the "Read:Org" scope selected:

![](PAT_creation.png)

You may choose to save the token in a file called token.txt, and read it in with the code below. Alternatively, just create a variable called ```token='<my_token>'```.

Note: You will not be able to get the token from the GitHub interface after it has been displayed for the first time, so it is recommended to save it somewhere (like a keychain).

In [2]:
def api_token():
    token_file = Path('token.txt')
    if not token_file.exists():
        raise IOError('Please create a token at github.com, and save it in {}'.format(token_file))

    token = token_file.read_text().strip()
    return token


token = api_token()

Now that we have a personal access token, let's define the headers that will allow us to use it:

In [3]:
headers = {'Authorization': 'token {}'.format(token),
           'Accept': 'application/vnd.github.v3+json'}
API_URL = 'https://api.github.com'

Finally, we are ready. Let's find out what the "login" (username) is for this personal access token.

In [32]:
# The endpoint target.
target = '/user'

# The actual HTTP request
resp = requests.get('{}{}'.format(API_URL, target),
                    headers=headers)

# Raise an error if we didn't get a 200 return code.
resp.raise_for_status()

# Assume the response is JSON, and get hold of it.
content = resp.json()

# Print the content so that we know what a GitHub response actually looks like.
pprint(content)

# Get hold of the user's login ID
login = content['login']

{'avatar_url': 'https://avatars3.githubusercontent.com/u/810663?v=4',
 'bio': 'Scientific software engineer and problem solver.',
 'blog': 'https://pelson.github.io',
 'company': None,
 'created_at': '2011-05-25T20:18:23Z',
 'email': 'pelson.pub@gmail.com',
 'events_url': 'https://api.github.com/users/pelson/events{/privacy}',
 'followers': 87,
 'followers_url': 'https://api.github.com/users/pelson/followers',
 'following': 23,
 'following_url': 'https://api.github.com/users/pelson/following{/other_user}',
 'gists_url': 'https://api.github.com/users/pelson/gists{/gist_id}',
 'gravatar_id': '',
 'hireable': True,
 'html_url': 'https://github.com/pelson',
 'id': 810663,
 'location': 'UK',
 'login': 'pelson',
 'name': 'Phil Elson',
 'organizations_url': 'https://api.github.com/users/pelson/orgs',
 'public_gists': 55,
 'public_repos': 168,
 'received_events_url': 'https://api.github.com/users/pelson/received_events',
 'repos_url': 'https://api.github.com/users/pelson/repos',
 'site_admin':

Now, let's look at all the repos that this user has access to:

In [8]:
target = '/user/repos'
resp = requests.get('{}{}'.format(API_URL, target),
                    headers=headers)
resp.raise_for_status()
content = resp.json()
pprint(content[0])

{'archive_url': 'https://api.github.com/repos/bblay/iris_logo/{archive_format}{/ref}',
 'assignees_url': 'https://api.github.com/repos/bblay/iris_logo/assignees{/user}',
 'blobs_url': 'https://api.github.com/repos/bblay/iris_logo/git/blobs{/sha}',
 'branches_url': 'https://api.github.com/repos/bblay/iris_logo/branches{/branch}',
 'clone_url': 'https://github.com/bblay/iris_logo.git',
 'collaborators_url': 'https://api.github.com/repos/bblay/iris_logo/collaborators{/collaborator}',
 'comments_url': 'https://api.github.com/repos/bblay/iris_logo/comments{/number}',
 'commits_url': 'https://api.github.com/repos/bblay/iris_logo/commits{/sha}',
 'compare_url': 'https://api.github.com/repos/bblay/iris_logo/compare/{base}...{head}',
 'contents_url': 'https://api.github.com/repos/bblay/iris_logo/contents/{+path}',
 'contributors_url': 'https://api.github.com/repos/bblay/iris_logo/contributors',
 'created_at': '2012-10-06T12:12:49Z',
 'default_branch': 'master',
 'deployments_url': 'https://api.

In [9]:
pprint([repo['full_name'] for repo in content][:5])

['bblay/iris_logo',
 'conda-forge/addict-feedstock',
 'conda-forge/ads-feedstock',
 'conda-forge/affine-feedstock',
 'conda-forge/agate-dbf-feedstock']


Similarly, let's get hold of all the repos in the user's GitHub account:

In [25]:
target = '/users/{}/repos'.format(login)
resp = requests.get('{}{}'.format(API_URL, target),
                    headers=headers)
resp.raise_for_status()
content = resp.json()

pprint([repo['full_name'] for repo in content][:5])

['pelson/anaconda-build',
 'pelson/anaconda-list-distributions',
 'pelson/anaconda-recipes',
 'pelson/antigrain',
 'pelson/artview-feedstock']


There are a number of repos starting with "a" in this user's account. Let's change the sort order by modifying the request:

In [28]:
target = '/users/{}/repos'.format(login)

params = {'sort': 'updated'}
resp = requests.get('{}{}'.format(API_URL, target),
                    params=arguments,
                    headers=headers)
resp.raise_for_status()
content = resp.json()

pprint([repo['full_name'] for repo in content][:5])

['pelson/intro-to-using-requests-with-github-API',
 'pelson/my-first-heroku-tornado-app',
 'pelson/example-supervisord-manager',
 'pelson/conda-rpms',
 'pelson/python-gnupg']


The GitHub API is an endless trove of information. For example, one can get hold of a number of events that the authenticated user did with:

In [31]:
target = '/users/{}/events'.format(login)

params = {'sort': 'updated'}
resp = requests.get('{}{}'.format(API_URL, target),
                    params=arguments,
                    headers=headers)
resp.raise_for_status()
content = resp.json()

pprint({event['type'] for event in content})

{'CreateEvent',
 'GollumEvent',
 'IssueCommentEvent',
 'PullRequestEvent',
 'PullRequestReviewCommentEvent',
 'PushEvent'}
